In [1]:
import numpy as np

nn_architecture = [
    {"input_dim": 2, "output_dim": 4, "activation": "relu"},
    {"input_dim": 4, "output_dim": 6, "activation": "relu"},
    {"input_dim": 6, "output_dim": 6, "activation": "relu"},
    {"input_dim": 6, "output_dim": 4, "activation": "relu"},
    {"input_dim": 4, "output_dim": 1, "activation": "sigmoid"},
]

def init_layers(nn_architecture, seed = 99):
    np.random.seed(seed)
    number_of_layers = len(nn_architecture)
    params_values = {}

    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx + 1
        layer_input_size = layer["input_dim"]
        layer_output_size = layer["output_dim"]
        
        params_values['W' + str(layer_idx)] = np.random.randn(
            layer_output_size, layer_input_size) 
        params_values['b' + str(layer_idx)] = np.random.randn(
            layer_output_size, 1) 
        
    return params_values

def sigmoid(Z):
    return 1/(1+np.exp(-Z))

def relu(Z):
    return np.maximum(0,Z)

def sigmoid_backward(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)

def relu_backward(dA, Z):
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0] = 0;
    return dZ;

<h1> Full Forward Propagation 

In [2]:
def single_layer_forward_propagation(A_prev, W_curr, b_curr, activation="relu"):
    Z_curr = np.dot(W_curr, A_prev) + b_curr
    
    if activation == "relu":
        activation_func = relu
    elif activation == "sigmoid":
        activation_func = sigmoid
    else:
        raise Exception('Non-supported activation function')
        
    return activation_func(Z_curr), Z_curr

def full_forward_propagation(X, params_values, nn_architecture):
    memory = {}
    A_curr = X
    
    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx + 1
        A_prev = A_curr
        
        activ_function_curr = layer["activation"]
        W_curr = params_values["W" + str(layer_idx)]
        
        b_curr = params_values["b" + str(layer_idx)]
        A_curr, Z_curr = single_layer_forward_propagation(A_prev, W_curr, b_curr, activ_function_curr)
        
        memory["A" + str(idx)] = A_prev
        memory["Z" + str(layer_idx)] = Z_curr
       
    return A_curr, memory

<h1> Loss Function

In [3]:
def get_cost_value(predictions,targets):
    # Retrieving number of samples in dataset
    samples_num = len(predictions)
    
    # Summing square differences between predicted and expected values
    accumulated_error = 0.0
    for prediction, target in zip(predictions, targets):
        accumulated_error += (prediction - target)**2
        
    # Calculating mean and dividing by 2
    mae_error = (1.0 / (2*samples_num)) * accumulated_error
    
    return mae_error

def get_accuracy_value(Y_hat, Y):
    return 100 - np.mean(np.abs(Y_hat - Y)) * 100

<h1> Backward Propagation

In [44]:
def single_layer_backward_propagation(dA_curr, W_curr, b_curr, Z_curr, A_prev, activation="relu"):
    m = A_prev.shape[1]
    
    if activation == "relu":
        backward_activation_func = relu_backward
    elif activation == "sigmoid":
        backward_activation_func = sigmoid_backward
    else:
        raise Exception('Non-supported activation function')
    
    dZ_curr = backward_activation_func(dA_curr, Z_curr)
    dW_curr = np.dot(dZ_curr, A_prev.T) / m
    db_curr = np.sum(dZ_curr, axis=1, keepdims=True) / m
    dA_prev = np.dot(W_curr.T, dZ_curr)

    return dA_prev, dW_curr, db_curr

def full_backward_propagation(Y_hat, Y, memory, params_values, nn_architecture):
    grads_values = {}
    m = Y.shape[1]
    Y = Y.reshape(Y_hat.shape)
    
    print("Y:"+str(Y))
    print("Y_hat:"+str(Y_hat))
   
    dA_prev = - (np.divide(Y, Y_hat) - np.divide(1 - Y, 1 - Y_hat));
    
    for layer_idx_prev, layer in reversed(list(enumerate(nn_architecture))):
        layer_idx_curr = layer_idx_prev + 1
        activ_function_curr = layer["activation"]
        
        dA_curr = dA_prev
        
        A_prev = memory["A" + str(layer_idx_prev)]
        Z_curr = memory["Z" + str(layer_idx_curr)]
        W_curr = params_values["W" + str(layer_idx_curr)]
        b_curr = params_values["b" + str(layer_idx_curr)]
        
        dA_prev, dW_curr, db_curr = single_layer_backward_propagation(
            dA_curr, W_curr, b_curr, Z_curr, A_prev, activ_function_curr)
        
        grads_values["dW" + str(layer_idx_curr)] = dW_curr
        grads_values["db" + str(layer_idx_curr)] = db_curr
    
    return grads_values

<h1> Updating

In [35]:
def update(params_values, grads_values, nn_architecture, learning_rate):
    for layer_idx, layer in enumerate(nn_architecture):
        params_values["W" + str(layer_idx+1)] -= learning_rate * grads_values["dW" + str(layer_idx+1)]        
        params_values["b" + str(layer_idx+1)] -= learning_rate * grads_values["db" + str(layer_idx+1)]

    return params_values;

<h1> Training the Network

In [36]:
def train(X, Y, nn_architecture, epochs, learning_rate):
    params_values = init_layers(nn_architecture, 2)
    cost_history = []
    accuracy_history = []
    
    for i in range(epochs):
        Y_hat, cashe = full_forward_propagation(X, params_values, nn_architecture)
        print("Yhat:"+str(Y_hat.shape))
        cost = get_cost_value(Y_hat, Y)
        print("Cost:"+str(cost))
        cost_history.append(cost)
        accuracy = get_accuracy_value(Y_hat, Y)
        accuracy_history.append(accuracy)
        
        grads_values = full_backward_propagation(Y_hat, Y, cashe, params_values, nn_architecture)
        params_values = update(params_values, grads_values, nn_architecture, learning_rate)
        
    return params_values, cost_history, accuracy_history

In [37]:
def real_function(xy_input):
    output = []
    for i in range(xy_input.shape[1]):
        output.append(sigmoid(xy_input[0,i]*xy_input[0,i] + xy_input[1,i]*xy_input[1,i]))
    
    return np.array(output)[np.newaxis,:]

def input_generator(samples):
    x = np.random.rand(samples)
    y = np.random.rand(samples)
    return np.vstack((x,y))


In [46]:
x = input_generator(3)
x = np.array([[0.5,1010,528],[0.851,506,152]])
y = real_function(x)
train(x,y,nn_architecture,1,0.1)

Yhat:(1, 3)
Cost:[0.12910682 0.44837558 0.5       ]
Y:[[0.72595605 1.         1.        ]]
Y_hat:[[2.17808795e-01 5.30305396e-02 4.83341001e-17]]


({'W1': array([[-0.41675785, -0.05626683],
         [-2.1361961 ,  1.64027081],
         [-1.79343559, -0.84174737],
         [-7.81808566, -3.64131731]]),
  'b1': array([[-1.05795222],
         [-0.90900761],
         [ 0.55145404],
         [ 2.2756018 ]]),
  'W2': array([[ 0.04153939, -1.11792545,  0.53905832, -0.5961597 ],
         [-0.0191305 ,  1.17500122, -0.74787095,  0.00902525],
         [-0.87810789, -0.15643417,  0.25657045, -0.98877905],
         [-0.33882197, -0.23618403, -0.63765501, -1.18761229],
         [-1.42121723, -0.1534952 , -0.26905696,  1.7057244 ],
         [-2.43476758,  0.1127265 ,  0.37044454,  1.31118185]]),
  'b2': array([[ 0.53303946],
         [-0.8442137 ],
         [-0.00940098],
         [ 0.4961636 ],
         [-0.31778684],
         [ 0.73302741]]),
  'W3': array([[-1.86809065,  1.73118467,  1.46767801, -0.33567734, -1.322441  ,
          -1.14265526],
         [-0.82913529,  0.08771022,  1.00036589, -0.38109252, -0.37566942,
          -0.07447076]

In [43]:
x = input_generator(3)
x = np.array([[1010],[506]])
y = real_function(x)
train(x,y,nn_architecture,1,0.1)

Yhat:(1, 1)
Cost:[0.44837558]
Y:[[1.]]
Y:[[0.05303054]]


({'W1': array([[-0.41675785, -0.05626683],
         [-2.1361961 ,  1.64027081],
         [-1.79343559, -0.84174737],
         [ 0.50288142, -1.24528809]]),
  'b1': array([[-1.05795222],
         [-0.90900761],
         [ 0.55145404],
         [ 2.29220801]]),
  'W2': array([[ 0.04153939, -1.11792545,  0.53905832, -0.5961597 ],
         [-0.0191305 ,  1.17500122, -0.74787095,  0.00902525],
         [-0.87810789, -0.15643417,  0.25657045, -0.98877905],
         [-0.33882197, -0.23618403, -0.63765501, -1.18761229],
         [-1.42121723, -0.1534952 , -0.26905696,  2.23136679],
         [-2.43476758,  0.1127265 ,  0.37044454,  1.35963386]]),
  'b2': array([[ 0.59540395],
         [-0.8442137 ],
         [-0.02822245],
         [ 0.40378565],
         [-0.3135082 ],
         [ 0.67263423]]),
  'W3': array([[-1.86809065,  1.73118467,  1.46767801, -0.33567734,  0.61134078,
           0.04797059],
         [-0.82913529,  0.08771022,  1.00036589, -0.38109252, -0.37566942,
          -0.07447076]

In [40]:
real_function(np.array([[0.5,52],[0.851,102]]))

array([[0.72595605, 1.        ]])

In [8]:
np.sum(np.array([4,5,6])[np.newaxis,:]-np.array([1,2,3])[np.newaxis,:])

9

In [9]:
np.array([4,5,6])[np.newaxis,:]

array([[4, 5, 6]])

In [23]:
print("hello")

hello
